In [102]:
import sys

sys.path.append("..")
import pickle as pkl
from copy import deepcopy
from dataclasses import asdict, is_dataclass
from itertools import product

import numpy as np
import pandas as pd
import plotly.express as px
import plotly.io as pio
from dacite import from_dict
from ipywidgets import interact
from scipy.spatial.transform import Rotation
from tqdm.auto import tqdm

from calibration.benchmark.benchmark import BenchmarkResult, benchmark_simul
from calibration.feature_detector.visualization import show_boards, show_corners
from calibration.projector.board import gen_charuco_grid, gen_checkerboard_grid
from calibration.projector.camera import Camera

# pio.renderers.default = "svg"
from calibration.projector.projector import Projector
from calibration.solver.solve import solve

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [94]:
with open("../simul_results.pkl", "rb") as f:
    results = pkl.load(f)

In [ ]:
ts = product([-0.1, 0.1], [-0.1, 0.1], np.arange(-0.2, -0.01, 0.01))
ts = list(map(np.array, ts))
Rs = [np.eye(3), Rotation.from_euler("z", 10, degrees=True).as_matrix()]
Rs = [np.eye(3)]
lambdass = [
    np.array([l1, l2])
    for l1 in np.arange(-1.5, 1.51, 1)
    for l2 in np.arange(
        -2.61752136752137 * l1 - 6.85141810943093,
        -2.61752136752137 * l1 - 4.39190876941320,
        0.5,
    )
]
cameras = [
    Camera(),
    # Camera(135.0, np.array([40, 30]), np.array([1920, 1080]), 1.0),
    Camera(135.0, np.array([40, 30]), np.array([1920, 1080])),
]
boards = [gen_checkerboard_grid(7, 9), gen_charuco_grid(7, 9, 0.4, 0.2)]

results = []
for t, R, lambdas, camera, board in product(ts, Rs, lambdass, cameras, boards):
    proj = Projector(R=R, t=t, lambdas=lambdas, camera=camera)
    try:
        x = proj.project(board)
        if not (x > 0).all():
            rez = 2
        elif not (x < proj.camera.resolution).all():
            rez = 3
        else:
            X_ = proj.backproject(x)
            if np.allclose(X_, board, atol=1e-10):
                rez = 0
            else:
                rez = 4

    except ValueError:
        rez = 1
    results.append(
        dict(
            R=R,
            t=t,
            lambdas=lambdas,
            camera=camera.focal_length == Camera().focal_length,
            rez=str(rez),
        )
    )

In [103]:
Rs = [
    np.eye(3),
    # Rotation.from_euler("z", 10, degrees=True).as_matrix(),
    Rotation.from_euler("xyz", [10, 10, 10], degrees=True).as_matrix(),
]
lambdass = [
    np.array([l1, l2])
    for l1 in np.arange(-1.5, 1.51, 1)
    for l2 in np.arange(
        -2.61752136752137 * l1 - 6.85141810943093,
        -2.61752136752137 * l1 - 4.39190876941320,
        0.5,
    )
]
cameras = [
    Camera(),
    Camera(135.0, np.array([40, 30]), np.array([1920, 1080])),
    Camera(135.0, np.array([40, 30]), np.array([1920, 1080]), 1.0),
]
ts_for_cameras = [
    list(map(np.array, product([-0.1, 0.1], [-0.1, 0.1], [-0.2, -0.01]))),
    list(map(np.array, product([-0.01, 0.01], [-0.01, 0.01], [-0.1, -0.01]))),
    list(map(np.array, product([-0.01, 0.01], [-0.01, 0.01], [-0.1, -0.01]))),
]

results = []
for camera, ts in zip(cameras, ts_for_cameras):
    for R, lambdas, t in tqdm(
        product(Rs, lambdass, ts),
        leave=False,
        total=len(Rs) * len(lambdass) * len(ts),
        desc="Testing benchmark",
    ):
        kwargs = dict(R=R, lambdas=lambdas, t=t, camera=camera)
        try:
            rr = benchmark_simul(100, kwargs=kwargs)
        except ValueError:
            rez = 1
        else:
            for k, v in kwargs.items():
                for r in rr:
                    if isinstance(v, np.ndarray):
                        np.testing.assert_array_equal(getattr(r.input, k), v)
                    else:
                        for f in fields(v):
                            if isinstance(getattr(v, f.name), np.ndarray):
                                np.testing.assert_array_equal(
                                    getattr(v, f.name), getattr(camera, f.name)
                                )
                            else:
                                assert getattr(v, f.name) == getattr(camera, f.name)

            assert None not in rr

Testing benchmark:   0%|          | 0/320 [00:00<?, ?it/s]

Generating projectors: 0it [00:00, ?it/s]

Simulating:   0%|          | 0/100 [00:00<?, ?it/s]

Calibrating:   0%|          | 0/100 [00:00<?, ?it/s]

Evaluating:   0%|          | 0/100 [00:00<?, ?it/s]

NameError: name 'fields' is not defined

In [118]:
kwargs

{'R': array([[ 0.96984631, -0.14131448,  0.19856573],
        [ 0.17101007,  0.97508244, -0.14131448],
        [-0.17364818,  0.17101007,  0.96984631]]),
 'lambdas': array([-1.5       , -1.92513606]),
 't': array([-0.01, -0.01, -0.1 ]),
 'camera': Camera(focal_length=135.0, sensor_size=array([40, 30]), resolution=array([1920, 1080]), skew=0.0)}

In [112]:
kwargs = dict(R=Rs[1], lambdas=lambdass[2], t=ts_for_cameras[1][0], camera=cameras[1])
rr = benchmark_simul(100, kwargs=kwargs)

Generating projectors: 0it [00:00, ?it/s]

Simulating:   0%|          | 0/100 [00:00<?, ?it/s]

Calibrating:   0%|          | 0/100 [00:00<?, ?it/s]

Evaluating:   0%|          | 0/100 [00:00<?, ?it/s]

In [115]:
proj = Projector(**kwargs)
X = gen_checkerboard_grid(7, 9)
x = proj.project(X)
proj_ = solve(x, X, proj.camera)
proj_.project(X)

ValueError: f(a) and f(b) must have different signs

In [116]:
proj

Projector(R=array([[ 0.96984631, -0.14131448,  0.19856573],
       [ 0.17101007,  0.97508244, -0.14131448],
       [-0.17364818,  0.17101007,  0.96984631]]), t=array([-0.01, -0.01, -0.1 ]), lambdas=array([-1.5       , -1.92513606]), camera=Camera(focal_length=135.0, sensor_size=array([40, 30]), resolution=array([1920, 1080]), skew=0.0))

In [117]:
proj_

Projector(R=array([[-0.94956828, -0.15702426, -0.27141015],
       [ 0.18672055, -0.97854127, -0.08713451],
       [-0.2519038 , -0.13341802,  0.9585114 ]]), t=array([0.11065925, 0.07918207, 8.94537215]), lambdas=array([ -54.27799501, 8643.5077533 ]), camera=Camera(focal_length=135.0, sensor_size=array([40, 30]), resolution=array([1920, 1080]), skew=0.0))